<a href="https://colab.research.google.com/github/j-river1/Webscraping-Elecciones-EstadosUnidos/blob/main/src/WebScrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Web Scrapping

Este codigo tiene como finalidad la descarga de los datos de https://ballotpedia.org/Voter_registration

# Instalación de librerias 

Para el webscrapping se utiliza la librerias beatifulSoup [1]

In [6]:
!pip install beautifulsoup4
!pip install urllib3
!pip install html-table-parser-python3
!pip install builtwith



ERROR: Operation cancelled by user
  Created wheel for builtwith: filename=builtwith-1.3.4-cp36-none-any.whl size=36100 sha256=7dea1ad4e577b800116a409837dbb4f5dfa0807585a2976799c9dad0316a4018
  Stored in directory: /root/.cache/pip/wheels/ab/bf/98/1e11b10a7c3bdea99ff8fe79abc7d3d22a254cab4753a88d52
Successfully built builtwith


In [7]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import requests
import bs4
import lxml.etree as xml
import urllib.request
from pprint import pprint
from html_table_parser import HTMLTableParser
import pandas as pd
import os
import builtwith


# Tecnología de la pagina 
En esta sección se muestra la tecnologia utilizada en la pagina web.

In [8]:
builtwith.builtwith('https://ballotpedia.org/Voter_registration')

{'analytics': ['Optimizely'],
 'font-scripts': ['Google Font API'],
 'web-frameworks': ['Twitter Bootstrap'],
 'web-servers': ['Apache'],
 'widgets': ['AddThis'],
 'wikis': ['MediaWiki']}

# Introducción

Este web scrapping tiene como objetivo descargar toda la información necesaria para votar en la elecciones presidenciales de los Estados Unidos. El contenido de esta información está en el sitio web **Ballotpedia**. El web scapping empieza por descargar la informacion de las tablas de este sitio web y luego se une una sola base de datos. La base de datos descargada tiene la siguiente información:




* **State**. Estado. 
* **All voters required to show ID**. Todos los votantes deben mostrar una identificación ("Sí" indica que todos los votantes deben presentar una identificación en las urnas). 
* **Select voters required to show ID**. Seleccione los votantes que deben mostrar una identificación (una entrada en este campo describe qué votantes deben mostrar una identificación en las urnas si no hay un requisito de identificación universal). 
* **Accepted IDs** Identificaciones aceptadas (una lista de formas aceptables de identificación). 
*  **Registration URL** URL de registro (un enlace al sitio web del estado donde los votantes elegibles pueden registrarse, si corresponde)

* **Registration status URL** URL de estado de registro (un enlace al sitio web del estado donde los votantes pueden verificar su estado de registro, si corresponde).

* **Registration update URL** URL de actualización de registro (un enlace al sitio web del estado donde los votantes pueden actualizar sus registros, si corresponde)

* **In-person registration deadline** Fecha límite de registro en persona (el último día que un votante puede registrarse en persona, si corresponde).

*  **Mail postmark or receipt deadline**. Fecha límite de registro por correo (el último día que un votante puede registrarse por correo, si corresponde)

* **Mail registration deadline**. Matasellos de correo o fecha límite de recepción (ya sea que la fecha límite de registro por correo sea un matasellos o fecha límite de recepción).

* **Online registration deadline** Fechas límite de registro en línea (el último día que un votante puede registrarse en línea, si corresponde)

* **Same-day registration** Registro el mismo día ("Sí" indica que el estado permite que los votantes se registren el día de las elecciones).

* **Early voting same-day registration** Inscripción anticipada el mismo día ("Sí" indica que el estado permite que los votantes se registren durante el período de votación anticipada, si corresponde).

* **Early voting start date** Fecha de inicio de la votación anticipada

* **Early voting end date** Fecha de finalización de la votación anticipada


A continuación está el codigo para la descarga de cada una de las tablas y de la unión de estas.

# Voter registration statistics


In [9]:
# Opens a website and read its
# binary contents (HTTP Response Body)
def url_get_contents(url):

    # Opens a website and read its
    # binary contents (HTTP Response Body)

    #making request to the website
    req = urllib.request.Request(url=url)
    f = urllib.request.urlopen(req)

    #reading contents of the website
    return f.read()

In [10]:
xhtml = url_get_contents('https://ballotpedia.org/Voter_registration').decode('utf-8')

# Defining the HTMLTableParser object
p = HTMLTableParser()

# feeding the html contents in the
# HTMLTableParser object
p.feed(xhtml)

# Now finally obtaining the data of
# the table required
dfes = pd.DataFrame(p.tables[1], columns= p.tables[1].pop(1))
dfes

,State,Total citizen population,Total registration,Registration as a percentage of citizen population
0,"Voter registration statistics by state, 2018",None,None,None
1,Alabama,"3,609,000","2,490,000",69.0%
2,Alaska,"497,000","337,000",67.7%
3,Arizona,"4,757,000","3,262,000",68.6%
4,Arkansas,"2,158,000","1,262,000",58.5%
5,California,"25,525,000","15,690,000",61.5%
6,Colorado,"4,029,000","2,645,000",65.6%
7,Connecticut,"2,539,000","1,726,000",68.0%
8,Delaware,"713,000","472,000",66.3%
9,District of Columbia,"512,000","397,000",77.6%


# Do I need ID to vote?

In [12]:
URL = "https://ballotpedia.org/Main_Page"
pagina = requests.get(URL)
soup = BeautifulSoup(pagina.content, 'html.parser')
selec = soup.select("a[href*=Voting_in_2020]")
link = selec[0].get('href')
pagina2 = requests.get(link)
soup2 = BeautifulSoup(pagina2.content)


table = soup2.find(lambda tag: tag.has_attr('id') and tag['id']=="VoterID") 
rows = table.findAll(lambda tag: tag.name=='tr')

tabla = []
for i in range(len(rows)):
  tabla.append(rows[i].get_text())

tabla_aux = []
for j in range(len(tabla)):
  tabla_aux.append(tabla[j].split("\n"))

 
column_names = tabla_aux.pop(0)
df = pd.DataFrame(tabla_aux, columns=column_names)
df6 = df[['State', 'All voters required to show ID', 'Select voters required to show ID', 'Accepted IDs']]
df6

,State,All voters required to show ID,Select voters required to show ID,Accepted IDs
0,Nevada,No,"""If a person’s name appears in the election bo...","""(a) The card issued to the voter at the time ..."
1,California,No,"""In most cases, a California voter is not requ...","""A copy of a recent utility bill, the sample b..."
2,Texas,Yes,,"""Texas Driver License issued by the Texas Depa..."
3,Colorado,Yes,,"""A valid Colorado driver’s license or valid id..."
4,American Samoa,Yes,,"""Any qualified elector who wishes to cast his/..."
5,Michigan,Yes,,"""Michigan driver's license or state-issued ID ..."
6,South Carolina,Yes,,"""SC driver's license (includes standard licens..."
7,Kansas,Yes,,"""Driver’s license or ID card issued by Kansas ..."
8,Illinois,No,"""If you register by mail, you must vote in per...","""A current and valid photo identification. Uti..."
9,West Virginia,Yes,,"""Voter registration card; Medicare card or Soc..."


# How do I register to vote?

In [13]:
table = soup2.find(lambda tag: tag.has_attr('id') and tag['id']=="VoterRegistration") 
rows = table.findAll(lambda tag: tag.name=='tr')

tabla = []
for i in range(len(rows)):
  tabla.append(rows[i].get_text())

tabla_aux = []
for j in range(len(tabla)):
  tabla_aux.append(tabla[j].split("\n"))

 
column_names = tabla_aux.pop(0)
print(column_names)
df = pd.DataFrame(tabla_aux, columns=column_names)
df7 = df[['State', 'Registration URL', 'Registration status URL', 'Registration update URL']]
df7


['', 'State', 'Registration URL', 'Registration status URL', 'Registration update URL', '']


,State,Registration URL,Registration status URL,Registration update URL
0,Nevada,https://www.nvsos.gov/sosvoterservices/Registr...,https://www.nvsos.gov/votersearch/,https://www.nvsos.gov/sosvoterservices/Registr...
1,California,https://registertovote.ca.gov/,https://voterstatus.sos.ca.gov/,https://registertovote.ca.gov/
2,Texas,,https://teamrv-mvp.sos.texas.gov/MVP/mvp.do,https://www.votetexas.gov/register-to-vote/did...
3,Colorado,,https://www.sos.state.co.us/voter/pages/pub/ol...,https://www.sos.state.co.us/voter/pages/pub/ol...
4,American Samoa,,,
5,Michigan,https://mvic.sos.state.mi.us/RegisterVoter,https://mvic.sos.state.mi.us/Voter/Index,https://mvic.sos.state.mi.us/RegisterVoter
6,South Carolina,https://info.scvotes.sc.gov/eng/ovr/start.aspx,https://info.scvotes.sc.gov/eng/voterinquiry/V...,https://www.scvotes.gov/update-your-voter-regi...
7,Kansas,https://www.kdor.ks.gov/apps/voterreg/default....,https://myvoteinfo.voteks.org/voterview,https://www.kdor.ks.gov/apps/voterreg/default....
8,Illinois,https://ova.elections.il.gov/,https://ova.elections.il.gov/RegistrationLooku...,https://ova.elections.il.gov/
9,West Virginia,,https://services.sos.wv.gov/Elections/Voter/Am...,https://ovr.sos.wv.gov/Register/Landing#Qualif...


# What is the deadline to register to vote?

In [14]:
table = soup2.find(lambda tag: tag.has_attr('id') and tag['id']=="RegistrationDeadlines") 
rows = table.findAll(lambda tag: tag.name=='tr')

tabla = []
for i in range(len(rows)):
  tabla.append(rows[i].get_text())

tabla_aux = []
for j in range(len(tabla)):
  tabla_aux.append(tabla[j].split("\n"))

 
column_names = tabla_aux.pop(0)
#print(column_names)
df = pd.DataFrame(tabla_aux, columns=column_names)
df8 = df[[ 'State', 'In-person registration deadline', 'Mail registration deadline', 'Mail postmark or receipt deadline', 'Online registration deadline']]
df8

,State,In-person registration deadline,Mail registration deadline,Mail postmark or receipt deadline,Online registration deadline
0,Nevada,"October 6, 2020","October 6, 2020",Postmarked,"October 29, 2020"
1,California,"November 3, 2020","October 19, 2020",Postmarked,"October 19, 2020"
2,Texas,"October 5, 2020","October 5, 2020",Postmarked,
3,Colorado,"November 3, 2020","October 26, 2020",Received,"October 26, 2020"
4,American Samoa,"October 5, 2020",,,
5,Michigan,"November 3, 2020","October 19, 2020",Postmarked,"October 19, 2020"
6,South Carolina,"October 2, 2020","October 5, 2020",Postmarked,"October 4, 2020"
7,Kansas,"October 13, 2020","October 13, 2020",Postmarked,"October 13, 2020"
8,Illinois,"November 3, 2020","October 6, 2020",Postmarked,"October 18, 2020"
9,West Virginia,"October 13, 2020","October 13, 2020",Postmarked,"October 13, 2020"


# Can I register to vote the same day that I vote?

In [15]:
table = soup2.find(lambda tag: tag.has_attr('id') and tag['id']=="SamedayRegistration") 
rows = table.findAll(lambda tag: tag.name=='tr')

tabla = []
for i in range(len(rows)):
  tabla.append(rows[i].get_text())

tabla_aux = []
for j in range(len(tabla)):
  tabla_aux.append(tabla[j].split("\n"))

 
column_names = tabla_aux.pop(0)
df = pd.DataFrame(tabla_aux, columns=column_names)
df9= df[[ 'State', 'Same-day registration ', 'Early voting same-day registration']]
df9

,State,Same-day registration,Early voting same-day registration
0,Nevada,Yes,Yes
1,California,Yes,Yes
2,Texas,No,No
3,Colorado,Yes,Yes
4,American Samoa,No,
5,Michigan,Yes,Yes
6,South Carolina,No,No
7,Kansas,No,No
8,Illinois,Yes,Yes
9,West Virginia,No,No


# Can I vote early?

In [16]:

table = soup2.find(lambda tag: tag.has_attr('id') and tag['id']=="EarlyVoting") 
rows = table.findAll(lambda tag: tag.name=='tr')

tabla = []
for i in range(len(rows)):
  tabla.append(rows[i].get_text())

tabla_aux = []
for j in range(len(tabla)):
  tabla_aux.append(tabla[j].split("\n"))

 
column_names = tabla_aux.pop(0)
print(column_names)
df = pd.DataFrame(tabla_aux, columns=column_names)
df10= df[[ 'State', 'Early voting start date', 'Early voting end date']]
df10

['', 'State', 'Early voting start date', 'Early voting end date', '']


,State,Early voting start date,Early voting end date
0,Nevada,"October 17, 2020","October 30, 2020"
1,California,"October 5, 2020","November 2, 2020"
2,Texas,"October 13, 2020","October 30, 2020"
3,Colorado,"October 19, 2020","November 2, 2020"
4,American Samoa,,
5,Michigan,"September 24, 2020","November 2, 2020"
6,South Carolina,"October 5, 2020","November 2, 2020"
7,Kansas,"October 14, 2020","November 2, 2020"
8,Illinois,"September 24, 2020","November 2, 2020"
9,West Virginia,"October 21, 2020","October 31, 2020"


# Union de las tablas y base de datos final

In [20]:
#Merge de todas las tablas

aux =df9.merge(df8, on='State', how='left')
aux3 = df6.merge(df7, on='State', how='left')
aux4 = aux3.merge(aux, on='State', how='left')
total = aux4.merge(df10, on='State', how='left')

total
total2 = total.merge(dfes, on='State', how='left')
total2

total2.to_csv('/content/drive/My Drive/Colab Notebooks/Web Scraping/dataset.csv')
print(total2.shape)



(56, 18)


# Referencias

[1] https://www.crummy.com/software/BeautifulSoup/bs4/doc/#quick-start